The `DEMO_KEY` should work for a small number of requests, or request a key at https://api.data.gov/signup/.

The first block requests the [document](https://www.regulations.gov/docket?D=DOI-2017-0002) about Bears Ear National Monument that elicited 1.5 million comments.

In [1]:
import requests

API_KEY = 'DEMO_KEY'
path = 'https://api.data.gov/regulations/v3/document.json'
query = {'documentId':'DOI-2017-0002-0001', 'api_key':API_KEY}
response = requests.get(path, params=query)

Extract data from the returned JSON object, which gets mapped to a Python dictionary called `doc`.

In [2]:
doc = response.json()
print('{}: {}'.format(
    doc['numItemsRecieved']['label'],
    doc['numItemsRecieved']['value'],
))

Number of Comments Received: 2839046


Initiate a new API query for public submission (PS) comments and print the dictionary keys in the response.

In [3]:
query = {
    'dktid': doc['docketId']['value'],
    'dct': 'PS',
    'api_key': API_KEY}
path = 'https://api.data.gov/regulations/v3/documents.json'
response = requests.get(path, params=query)
dkt = response.json()
list(dkt.keys())

['documents', 'totalNumRecords']

The purported claimed number of results is much larger than the length of the documents array contained in this response.

In [ ]:
print('Number received: {}\nTotal number: {}'.format(
    len(dkt['documents']),
    dkt['totalNumRecords'],
))

We'll have to flip through "pages" with the API and store the response at each iteration.

The next block runs a system command to initialize an empty database called "BENM", this can only be done once and would typically be performed by a database administrator (which may, in fact, be you!).

In [2]:
from subprocess import check_output
check_output('echo $(whoami) | sudo -u postgres -S createdb BENM', shell=True) == b''

True

The following commands prepare Python to connect to the database, and creates empty tables in the database if they do not already exist (i.e. it is safe to re-run after you have populated the database).

**Step 1: Boilerplate**

In [ ]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine, Column
from sqlalchemy.dialects.postgresql import BIGINT, JSONB

Base = declarative_base()

**Step 2: Table Definition**

In [ ]:
class Comment(Base):
    __tablename__ = 'comment'
    
    id = Column(BIGINT, primary_key=True)
    json = Column(JSONB)

**Step 3: Connect and initialize**

In [ ]:
engine = create_engine('postgresql+pygresql://@localhost/BENM')
Session = sessionmaker(bind=engine)
Base.metadata.create_all(engine)

You could inspect the BENM database hosted by "localhost" now in the PostgreSQL Studio app; you would find one empty "comment" table with fields "id" and "json".

Add a new `rpp` parameter to request `100` documents per page.

In [ ]:
query = {
    'dktid': doc['docketId']['value'],
    'dct': 'PS',
    'rpp': 100,
    'api_key': API_KEY,
    }

In each request, advance the query parameter `po` to the number of the record you want the response to begin with. Insert the documents (the key:value pairs stored in `values`) in bulk to the database with `engine.execute()`.

In [ ]:
for i in range(0, 10):
    query['po'] = i * query['rpp']
    print(query['po'])
    response = requests.get(path, params=query)
    page = response.json()
    docs = page['documents']
    values = [{'json': json} for json in docs]
    insert = Comment.__table__.insert().values(values)
    engine.execute(insert)

See what the first document in the database looks like. Or set `id` to a different value to look at other comments in the database.

In [ ]:
s = Session()
id = 1
comment = s.query(Comment).filter_by(id=id).first()
print('id: {}\ntext: {}'.format(
    comment.id,
    comment.json['commentText']
))
s.close()

Notice that some of the comments appear to be identical, probably some form e-mail the public was pasting into the comment field.

In [ ]:
s = Session()
q = s.query(Comment.id, Comment.json['commentText'].label('text')).limit(30);
for r in q:
    print('{}: {}'.format(r.id, r.text[:75].replace('\n', ' ')))
s.close()

Begin pre-processing the texts. In the block below we rely on some "in-database" operations on the assumption that the data are too big to read into memory for processing in our Python process. The product is a new database table called `doc` with the following fields:

1. A new `id` for each unique comment text.
1. The number of duplicated comments per each `id`, labeled `rep`.
1. A `tokens` field containing a condensed version of the comment text as a list of stemmed words, excluding stopwords, and their positions in the comment.

In [ ]:
s = Session()
s.execute('''
    WITH dedupe AS (
        SELECT count(*) AS rep, to_tsvector(json ->> 'commentText') AS tokens
        FROM comment
        GROUP BY tokens)
    SELECT row_number() OVER (ORDER BY rep) AS id, rep, tokens
    INTO TABLE doc
    FROM dedupe;
    ''')
s.commit()

For our final step in ETL (the "Load" step), we do an in-database conversion of the data into an easier form for analysis. The new table `word` will have the `id` from the `doc` table, each `word` in that document and its `freq`.

In [ ]:
s.execute('''
    SELECT id AS doc_id, word, nentry AS freq
    INTO TABLE word
    FROM doc
    JOIN ts_stat('select tokens from doc where id = ' || id) ON true;
''')
s.commit()
s.close()

Don't forget to disconnect from your database!

In [ ]:
engine.dispose()